# Homework 1

https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md

In [6]:
import openai
from openai import OpenAI
from dotenv import load_dotenv
import os
import requests 
from elasticsearch import Elasticsearch
from tqdm.auto import notebook_tqdm
import tiktoken

In [7]:
load_dotenv()

client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [9]:

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [10]:
es_client = Elasticsearch('http://localhost:9200')

In [11]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'course-questions'}

In [12]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [13]:
for doc in notebook_tqdm(documents):
    es_client.index(index=index_name, body=doc)

100%|██████████| 948/948 [00:05<00:00, 167.29it/s]


In [14]:
query = 'How do I execute a command in a running docker container?'

In [15]:
def elastic_search_Q3(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    print(response)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append({
            'document': hit['_source'],
            'score': hit['_score']
        })
        
    return result_docs


In [16]:
resultsQ3 = elastic_search_Q3(query)
resultsQ3

{'took': 7, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 750, 'relation': 'eq'}, 'max_score': 82.021484, 'hits': [{'_index': 'course-questions', '_id': 'rNBvapAB2AO4z5OlRff3', '_score': 82.021484, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'utBvapAB2AO4z5OlOvWS', '_score': 75.19232, '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\

[{'document': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'},
  'score': 82.021484},
 {'document': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi

In [17]:
resultsQ3[0]['score']

82.021484

In [18]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    print(response)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
        
    return result_docs


In [19]:
search_results = elastic_search(query)
search_results

{'took': 4, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 345, 'relation': 'eq'}, 'max_score': 82.021484, 'hits': [{'_index': 'course-questions', '_id': 'rNBvapAB2AO4z5OlRff3', '_score': 82.021484, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'y9BvapAB2AO4z5OlRvei', '_score': 50.216507, '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or dir

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [20]:
search_results[2]['question']

'How do I copy files from a different folder into docker container’s working directory?'

In [21]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role":"user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [22]:
def build_prompt(query, search_results):
    
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION. 

    QUESTION: {question}

    CONTEXT: 
    {context}
    """.strip()
    
    context = ""
    
    for doc in search_results:
        context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question = query, context = context).strip()
    
    return prompt

In [23]:
prompt = build_prompt(query, search_results)
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION. 

    QUESTION: How do I execute a command in a running docker container?

    CONTEXT: 
    Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_dire

In [24]:
length_of_prompt = len(prompt)
print("Length of the prompt:", length_of_prompt)

Length of the prompt: 1481


In [25]:
encoding = tiktoken.encoding_for_model("gpt-4o")
number_of_tokens = len(encoding.encode(prompt))
print("Number of tokens:", number_of_tokens)

Number of tokens: 329


## Bonus: generating the answer 

In [26]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [27]:
rag(query)

{'took': 6, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 345, 'relation': 'eq'}, 'max_score': 84.050095, 'hits': [{'_index': 'course-questions', '_id': 'rNBvapAB2AO4z5OlRff3', '_score': 84.050095, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'y9BvapAB2AO4z5OlRvei', '_score': 51.04628, '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or dire

'To execute a command in a running Docker container, you can use the `docker exec` command. First, you need to find the container ID using the `docker ps` command, then execute your desired command. Here’s how you can do it:\n\n1. Find the container ID:\n    ```sh\n    docker ps\n    ```\n\n2. Execute a command, such as opening a bash shell, in the specific container:\n    ```sh\n    docker exec -it <container-id> bash\n    ```\n\nReplace `<container-id>` with the actual ID of the running container.'

## Bonus: calculating the costs

In [29]:
def calculate_cost(input_tokens_per_request, output_tokens_per_request, num_requests, input_cost_per_k, output_cost_per_k):
    
    # Total tokens to thousands
    total_input_tokens_k = (input_tokens_per_request * num_requests) / 1000
    total_output_tokens_k = (output_tokens_per_request * num_requests) / 1000
    
    # Costs for tokens
    cost_input = total_input_tokens_k * input_cost_per_k
    cost_output = total_output_tokens_k * output_cost_per_k
    
    # Total cost
    total_cost = cost_input + cost_output
    
    return total_cost

avg_input_tokens_per_request = 150
avg_output_tokens_per_request = 250
num_requests = 1000

# Cost per 1K tokens
input_cost_per_k = 0.005
output_cost_per_k = 0.015 

average_cost = calculate_cost(avg_input_tokens_per_request, avg_output_tokens_per_request, num_requests, input_cost_per_k, output_cost_per_k)
print(f"Total cost for average token usage (150 input, 250 output) for 1000 requests: ${average_cost:.3f}")

# Values from Q6 and Q7
input_tokens_Q6 = 1481 # length of the prompt
output_tokens_Q7 = 329

specific_cost = calculate_cost(input_tokens_Q6, output_tokens_Q7, num_requests, input_cost_per_k, output_cost_per_k)
print(f"Total cost for 1481 input tokens and 329 output tokens for 1000 requests: ${specific_cost:.3f}")


Total cost for average token usage (150 input, 250 output) for 1000 requests: $4.500
Total cost for 1481 input tokens and 329 output tokens for 1000 requests: $12.340
